In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
# Read data
mnist = input_data.read_data_sets("MNIST_data/")

In [2]:
image_size = 28
learning_rate = 0.0001
batch_size = 20 # no need to change
dimensionality = image_size*image_size
#Lambda = 0.0 

iter_num = 20 # number of iterations of alternating scheme
steps_number = 10000 # number of gradient steps

code_size = 60 #dimension of code
k = 30 #needed dimension code_size = 2k k=10,20
gamma = 10.0 # smoothness of manifold
mu = 10.0 # main parameter mu=10,20,40,80,160
epsilon = 0.1

In [3]:
# Define placeholders
training_data = tf.placeholder(tf.float32, [None, dimensionality])
gradient_training_data = tf.placeholder(tf.float32, [None, dimensionality])
old_P = tf.placeholder(tf.float32, shape=[None, dimensionality, dimensionality])
old_W = tf.placeholder(tf.float32, shape=[dimensionality, code_size])
old_b = tf.placeholder(tf.float32, shape=[code_size])
old_b_r = tf.placeholder(tf.float32, shape=[dimensionality])

In [4]:
# Variables to be tuned
W = tf.Variable(tf.truncated_normal([dimensionality, code_size], stddev=0.1))
b = tf.Variable(tf.constant(0.1, shape=[code_size]))

b_r = tf.Variable(tf.constant(0.1, shape=[dimensionality]))

code_data = tf.nn.sigmoid(tf.matmul(training_data, W) + b)
recover = tf.matmul(code_data, tf.transpose(W)) + b_r
grad_phi_psi = []
for i in range(batch_size):
    for j in range(dimensionality):
        grad_phi_psi.append(tf.gradients(recover[i][j], [training_data[i]], unconnected_gradients='zero')[0])
grad_phi_psi = tf.reshape(tf.stack(grad_phi_psi), [batch_size, dimensionality, dimensionality])


# this is gradient field close to our points
rand_training_data = training_data + tf.random.normal(shape=[batch_size, dimensionality],
                                                      mean=0.0,stddev=epsilon)
rand_code_data = tf.nn.sigmoid(tf.matmul(rand_training_data, W) + b)
rand_recover = tf.matmul(rand_code_data, tf.transpose(W)) + b_r
rand_grad_phi_psi = []
for i in range(batch_size):
    for j in range(dimensionality):
        rand_grad_phi_psi.append(tf.gradients(rand_recover[i][j], [rand_training_data[i]], unconnected_gradients='zero')[0])
rand_grad_phi_psi = tf.reshape(tf.stack(rand_grad_phi_psi), [batch_size, dimensionality, dimensionality])

new_code_data = tf.nn.sigmoid(tf.matmul(gradient_training_data, W) + b)
new_recover = tf.matmul(new_code_data, tf.transpose(W)) + b_r
new_grad_phi_psi = []
for i in range(batch_size):
    for j in range(dimensionality):
        new_grad_phi_psi.append(tf.gradients(new_recover[i][j], [gradient_training_data[i]], unconnected_gradients='zero')[0])
new_grad_phi_psi = tf.reshape(tf.stack(new_grad_phi_psi), [batch_size, dimensionality, dimensionality])

In [5]:
# Define the loss function
loss = tf.reduce_mean(tf.square(training_data - recover)) + \
       gamma*tf.reduce_mean(tf.square(grad_phi_psi-rand_grad_phi_psi)) + \
       mu*tf.reduce_mean(tf.square(new_grad_phi_psi - old_P))

#+ Lambda*tf.reduce_mean(tf.square(grad_phi_psi))

# Training step
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [6]:
# Run the training
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [7]:
N = 50000 # x_train.shape[0]=55000
x_train = mnist.train.images[:N]
N_grad = 1000
grad_x_train = mnist.train.images[:N_grad]

cur_U = np.zeros((N_grad, dimensionality, k))
cur_Sigma = np.zeros((N_grad, k, k))
cur_V = np.zeros((N_grad, k, dimensionality))
feed_P = np.zeros((batch_size, dimensionality, dimensionality))
cur_W = np.random.normal(0, 0.35, (dimensionality, code_size))
cur_b = np.zeros((code_size))
cur_b_r = np.zeros((dimensionality))

num_batches = int(N/batch_size)
grad_num_batches = int(N_grad/batch_size)

for iter in range(iter_num):
    for i in range(steps_number):
        # Get the next batch
        which_batch = i%num_batches
        input_batch = x_train[which_batch*batch_size:(which_batch+1)*batch_size]
        grad_which_batch = i%grad_num_batches
        grad_input_batch = grad_x_train[grad_which_batch*batch_size:(grad_which_batch+1)*batch_size]
        for r in range(batch_size):
            U = cur_U[grad_which_batch*batch_size+r]
            Sigma = cur_Sigma[grad_which_batch*batch_size+r]
            V = cur_V[grad_which_batch*batch_size+r]
            feed_P[r] = np.matmul(U,np.matmul(Sigma,V))
        feed_dict = {training_data: input_batch, gradient_training_data: grad_input_batch, 
                     old_P:feed_P,
                  old_W:cur_W, old_b:cur_b, old_b_r:cur_b_r}
        # Run the training step
        train_step.run(feed_dict=feed_dict)
        # Print the accuracy progress on the batch every 1000 steps
        if i%1000 == 0:
            train_accuracy = sess.run(loss, feed_dict=feed_dict)
            print("Step %d, training batch accuracy %g %%"%(i, train_accuracy*100))
    for grad_which_batch in range(grad_num_batches):
        grad_input_batch = grad_x_train[grad_which_batch*batch_size:(grad_which_batch+1)*batch_size]
        feed_dict = {gradient_training_data: grad_input_batch}
        local_grad = sess.run(new_grad_phi_psi, feed_dict=feed_dict)
        for r in range(batch_size):
            u, s, vh = np.linalg.svd(local_grad[r,:,:], full_matrices=True)
            cur_U[grad_which_batch*batch_size+r] = u[:,0:k:1]
            cur_V[grad_which_batch*batch_size+r] = np.transpose(vh[:,0:k:1])
            cur_Sigma[grad_which_batch*batch_size+r] = np.diag(s[0:k:1])
    [cur_W, cur_b, cur_b_r] = sess.run([W, b, b_r])


Step 0, training batch accuracy 23.0096 %
Step 1000, training batch accuracy 5.4452 %
Step 2000, training batch accuracy 4.18013 %
Step 3000, training batch accuracy 3.67042 %
Step 4000, training batch accuracy 3.0902 %
Step 5000, training batch accuracy 2.97673 %
Step 6000, training batch accuracy 2.93121 %
Step 7000, training batch accuracy 2.53083 %
Step 8000, training batch accuracy 2.42085 %
Step 9000, training batch accuracy 2.12662 %
Step 0, training batch accuracy 2.12482 %
Step 1000, training batch accuracy 2.27709 %
Step 2000, training batch accuracy 2.0222 %
Step 3000, training batch accuracy 1.96567 %
Step 4000, training batch accuracy 1.70039 %
Step 5000, training batch accuracy 1.75678 %
Step 6000, training batch accuracy 1.95421 %
Step 7000, training batch accuracy 1.76499 %
Step 8000, training batch accuracy 1.7339 %
Step 9000, training batch accuracy 1.49447 %
Step 0, training batch accuracy 1.57122 %
Step 1000, training batch accuracy 1.74265 %
Step 2000, training bat

Step 4000, training batch accuracy 1.0477 %
Step 5000, training batch accuracy 1.1605 %
Step 6000, training batch accuracy 1.26649 %
Step 7000, training batch accuracy 1.21188 %
Step 8000, training batch accuracy 1.2254 %
Step 9000, training batch accuracy 1.04547 %
Step 0, training batch accuracy 1.15975 %
Step 1000, training batch accuracy 1.26374 %
Step 2000, training batch accuracy 1.20986 %
Step 3000, training batch accuracy 1.22225 %
Step 4000, training batch accuracy 1.04334 %
Step 5000, training batch accuracy 1.159 %
Step 6000, training batch accuracy 1.26095 %
Step 7000, training batch accuracy 1.20787 %
Step 8000, training batch accuracy 1.21915 %
Step 9000, training batch accuracy 1.04131 %


In [8]:
def euclidean_distance(img_a, img_b):
    '''Finds the distance between 2 images: img_a, img_b'''
    # element-wise computations are automatically handled by numpy
    return sum((img_a - img_b) ** 2)

from collections import defaultdict

def find_majority(labels):
    '''Finds the majority class/label out of the given labels'''
    # defaultdict(type) is to automatically add new keys without throwing error.
    counter = defaultdict(int)
    for label in labels:
        counter[label] += 1

    # Finding the majority class.
    majority_count = max(counter.values())
    for key, value in counter.items():
        if value == majority_count:
            return key
train_images = np.asarray(mnist.train.images[:5000])
train_labels = np.asarray(mnist.train.labels[:5000])
test_images = np.asarray(mnist.test.images)
test_labels = np.asarray(mnist.test.labels)

def sigmoid(x):
    return 1. /(1+np.exp(-x))
def new_euclidean_distance(img_a, img_b):
    img_a = np.reshape(img_a, (1,-1))
    img_b = np.reshape(img_b, (1,-1))
    img_a = sigmoid(np.matmul(img_a, cur_W) + cur_b)
    img_b = sigmoid(np.matmul(img_b, cur_W) + cur_b)
    return np.sum((img_a - img_b) ** 2)

def new_predict(k, train_images, train_labels, test_images):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(new_euclidean_distance(test_image, image), label)
                    for (image, label) in zip(train_images, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return find_majority(k_labels)

# Predicting and printing the accuracy
i = 0
total_correct = 0
for test_image in test_images[:5000]:
    pred = new_predict(10, train_images, train_labels, test_image)
    if pred == test_labels[i]:
        total_correct += 1
    acc = (total_correct / (i+1)) * 100
    if i%100 == 0:
        print('test image['+str(i)+']', '\tpred:', pred, '\torig:', test_labels[i], '\tacc:', str(round(acc, 2))+'%')
    i += 1

test image[0] 	pred: 7 	orig: 7 	acc: 100.0%
test image[100] 	pred: 6 	orig: 6 	acc: 97.03%
test image[200] 	pred: 3 	orig: 3 	acc: 96.02%
test image[300] 	pred: 1 	orig: 4 	acc: 94.02%
test image[400] 	pred: 2 	orig: 2 	acc: 92.52%
test image[500] 	pred: 3 	orig: 3 	acc: 91.62%
test image[600] 	pred: 6 	orig: 6 	acc: 91.18%
test image[700] 	pred: 1 	orig: 1 	acc: 90.87%
test image[800] 	pred: 8 	orig: 8 	acc: 90.89%
test image[900] 	pred: 1 	orig: 1 	acc: 91.12%
test image[1000] 	pred: 9 	orig: 9 	acc: 91.01%
test image[1100] 	pred: 7 	orig: 7 	acc: 91.1%
test image[1200] 	pred: 3 	orig: 8 	acc: 90.84%
test image[1300] 	pred: 4 	orig: 4 	acc: 90.55%
test image[1400] 	pred: 6 	orig: 6 	acc: 90.58%
test image[1500] 	pred: 1 	orig: 7 	acc: 90.67%
test image[1600] 	pred: 3 	orig: 3 	acc: 90.69%
test image[1700] 	pred: 0 	orig: 0 	acc: 90.95%
test image[1800] 	pred: 6 	orig: 6 	acc: 90.73%
test image[1900] 	pred: 1 	orig: 1 	acc: 90.79%
test image[2000] 	pred: 6 	orig: 6 	acc: 90.95%
test 

In [9]:
def predict(k, train_images, train_labels, test_images):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_image, image), label)
                    for (image, label) in zip(train_images, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return find_majority(k_labels)

# Predicting and printing the accuracy
i = 0
total_correct = 0
for test_image in test_images[:5000]:
    pred = predict(10, train_images, train_labels, test_image)
    if pred == test_labels[i]:
        total_correct += 1
    acc = (total_correct / (i+1)) * 100
    if i%100 == 0:
        print('test image['+str(i)+']', '\tpred:', pred, '\torig:', test_labels[i], '\tacc:', str(round(acc, 2))+'%')
    i += 1

test image[0] 	pred: 7 	orig: 7 	acc: 100.0%
test image[100] 	pred: 6 	orig: 6 	acc: 96.04%
test image[200] 	pred: 3 	orig: 3 	acc: 94.53%
test image[300] 	pred: 1 	orig: 4 	acc: 93.36%
test image[400] 	pred: 2 	orig: 2 	acc: 92.02%
test image[500] 	pred: 3 	orig: 3 	acc: 91.62%
test image[600] 	pred: 6 	orig: 6 	acc: 90.68%
test image[700] 	pred: 1 	orig: 1 	acc: 90.16%
test image[800] 	pred: 8 	orig: 8 	acc: 90.14%
test image[900] 	pred: 1 	orig: 1 	acc: 90.46%
test image[1000] 	pred: 9 	orig: 9 	acc: 90.01%
test image[1100] 	pred: 7 	orig: 7 	acc: 90.19%
test image[1200] 	pred: 3 	orig: 8 	acc: 90.01%
test image[1300] 	pred: 4 	orig: 4 	acc: 89.62%
test image[1400] 	pred: 6 	orig: 6 	acc: 89.58%
test image[1500] 	pred: 1 	orig: 7 	acc: 89.54%
test image[1600] 	pred: 3 	orig: 3 	acc: 89.57%
test image[1700] 	pred: 0 	orig: 0 	acc: 89.65%
test image[1800] 	pred: 6 	orig: 6 	acc: 89.62%
test image[1900] 	pred: 1 	orig: 1 	acc: 89.69%
test image[2000] 	pred: 6 	orig: 6 	acc: 89.81%
test